In [1]:
import csv
import statistics

In [ ]:
# Assumptions and To Dos:
# - Missing data is replaced with a 0. This should be replaced with a surrounding "average" 
#   or somehow disregarded in calculations

# Other assumptions are written under each question

# Note
# This is so difficult to do without pandas!
# I am largely relying on list locations to perform analysis

In [2]:
# change as needed
abs_path = '/Users/annelise/sandbox/co_real_estate'
residence = 'Neighborhood_Zhvi_SingleFamilyResidence.csv'
rentals = 'Neighborhood_Zri_SingleFamilyResidenceRental.csv'

In [5]:
def get_data(type: str, state: str='CO', county: str=None) -> tuple[dict, dict, dict]:
    """
    Queries the appropriate file (residence of rentals) for the specified state or country
    
    Returns three dictionaries:
    metadata: the information about the RegionID
    data: the regionID and values for each month; 0 is inputted for missing data for now
    iloc: a way to lookup the region from the original `iloc` """
    
    temp_data = []
    data = {}
    metadata = {}
    iloc_dict = {}

    assert type in ['residence', 'rentals']

    # read file
    with open(f'{abs_path}/data/{eval(type)}', 'r') as file:
        csv_reader = csv.DictReader(file, delimiter=',')
        for row in csv_reader:
            temp_data.append(row)

    # rows to clean
    string_rows = ['RegionID', 'RegionName', 'City', 'State', 'Metro', 'CountyName']
    rank_rows = ['SizeRank']
    # make default empty list for each "column"
    keys = string_rows + rank_rows
    for key in keys:
        metadata.setdefault(key, [])

    # merge dictionaries into 1 dictionary with structure {key1: [value1, value2, value3], key2: [a1, a2, a3]}
    # assuming zeroes for missing data
    # filter out for appropriate state/county data
    iloc_count = 0
    for dictionary in temp_data:
        if dictionary['State'] != state:
            continue
        if county:
            if dictionary['CountyName'] != county:
                continue
        iloc_dict[iloc_count] = dictionary['RegionID'] + ' ' + dictionary['RegionName'] +' ' + dictionary['City']+' '+dictionary['Metro']
        iloc_count += 1
        data[dictionary['RegionID']] = []
        for k, v in dictionary.items():
            if k not in string_rows and k not in rank_rows:
                v = int(v) if v else 0
                data[dictionary['RegionID']] = data[dictionary['RegionID']] + [v]
            if k in keys:
                metadata[k] = metadata[k] + [v]

    return metadata, data, iloc_dict

In [6]:
# Query the data to answer the required questions
colorado_residence_metadata, colorado_residence_data, colorado_residence_iloc = get_data('residence', 'CO')
boulder_residence_metadata, boulder_residence_data, boulder_residence_iloc = get_data('residence', 'CO', 'Boulder')
colorado_rentals_metadata, colorado_rentals_data, colorado_rentals_iloc = get_data('rentals', 'CO')



In [8]:
# 1. Using the most recent data available in these csv files, what neighborhoods in Colorado have the highest median home value?`


# Assuming most recent data = 3 years, 36 months
months_lookback = 36
medians = {}
for k, v in colorado_residence_data.items():
    medians[k] = statistics.median(v[-months_lookback:])

# top five medians
find_medians = list(medians.values())
find_medians.sort(reverse=True)
max_medians = find_medians[0:5]

max_median_regions = []
print('The top 5 neighborhoods in Colorado that have the highest median home value in the last 3 years are: ')
for i in max_medians:
    n = list(medians.values()).index(i)
    print(f'{colorado_residence_iloc[n]} with median value ${i}')

The top 5 neighborhoods in Colorado that have the highest median home value in the last 3 years are: 
416092 Mapleton Hill Boulder Boulder with median value $1392100.0
268646 Country Club Denver Denver with median value $1256250.0
416076 East Chautauqua Boulder Boulder with median value $1209700.0
416099 Newlands Boulder Boulder with median value $1099300.0
416069 Shanahan Ridge Boulder Boulder with median value $1085850.0


In [18]:
# 2. What are the top 5 neighborhoods in Colorado, ranked by how much the home price has increased (%) over the last two years.
months_lookback = 24
increases = {}
for k, v in colorado_residence_data.items():
    increases[k] = (v[-1]-v[-months_lookback])/v[-months_lookback]
    
# top five increases
find_increases = list(increases.values())
find_increases.sort(reverse=True)
max_increases = find_increases[0:5]

max_increase_regions = []
print('The top 5 neighborhoods in Colorado that have the largest increases (%) over the last two years are: ')
for i in max_increases:
    n = list(increases.values()).index(i)
    pct = '{0:.4g}'.format(i*100)
    print(f'{colorado_residence_iloc[n]} with percentage increase {pct}%')
    

The top 5 neighborhoods in Colorado that have the largest increases (%) over the last two years are: 
268633 Chaffee Park Denver Denver with percentage increase 34.31%
268659 Elyria Swansea Denver Denver with percentage increase 34.2%
268768 Villa Park Denver Denver with percentage increase 34.14%
268614 Barnum Denver Denver with percentage increase 32.47%
403509 College View - South Platte Denver Denver with percentage increase 32.39%


In [23]:
# 3. What neighborhoods (top 5 again) in Colorado have the highest ratio of rental income to home value. 

# Assumptions:
# This ratio should be current so:
# Using the average of the last 12 months of data for both the rental and residences

months_lookback = 12
residences = {}
rentals = {}
rental_res_ratio = {}

for k, v in colorado_residence_data.items():
    residences[k] = statistics.mean(v[-months_lookback:])

for k, v in colorado_rentals_data.items():
    rentals[k] = statistics.mean(v[-months_lookback:])
    
# make ratios:
for k, v in residences.items():
    if rentals.get(k):
        rental_res_ratio[k] = rentals[k]/residences[k]
    else:
        rental_res_ratio[k] = 0
        
# top five ratios
find_ratios = list(rental_res_ratio.values())
find_ratios.sort(reverse=True)
max_ratios = find_ratios[0:5]

max_ratio_regions = []
print('The top 5 neighborhoods in Colorado that have the largest rental to home value ratio over the last 12 months are: ')
for i in max_ratios:
    n = list(rental_res_ratio.values()).index(i)
    ratio = '{0:.4g}'.format(i*100)
    print(f'{colorado_residence_iloc[n]} with ratio {ratio}')
    

The top 5 neighborhoods in Colorado that have the largest rental to home value ratio over the last 12 months are: 
268664 Globeville Denver Denver with ratio 0.6779
6018 Montbello Denver Denver with ratio 0.665
268659 Elyria Swansea Denver Denver with ratio 0.6644
342737 North Aurora Aurora Denver with ratio 0.6615
275471 Southeast Colorado Springs Colorado Springs Colorado Springs with ratio 0.6572


In [32]:
# 4. A real-estate investor has come to you with money to invest in Boulder County. 
# What neighborhood would you suggest she invest in? Use the data to justify your answer 
# and provide a quick estimate of her ROI after 3 years. 

# For simplicity and because I'm running out of time,
# I used a similar method as #2, looking at the total % increase for the past n months.

# For the ROI part of the question, I would use the % increase for the past n months, again because I'm out of time.

months_lookback = 24
increases = {}
for k, v in boulder_residence_data.items():
    increases[k] = (v[-1]-v[-months_lookback])/v[-months_lookback]
    
# top five increases
find_increases = list(increases.values())
find_increases.sort(reverse=True)
max_increases = find_increases[0:5]

max_increase_regions = []
print('The top 5 neighborhoods in Boulder might be good investments that have the largest increases (%) over the last two years are: ')
for i in max_increases:
    n = list(increases.values()).index(i)
    pct = '{0:.4g}'.format(i*100)
    print(f'{boulder_residence_iloc[n]} with percentage increase {pct}%')
    
print("\n")
print("I would suggest she invest in the fastest growing neighborhood in the past 24 months, Mapleton Hill which had an ROI in the past 2 years of 19%. However, it does depend on her budget.")
    

The top 5 neighborhoods in Boulder might be good investments that have the largest increases (%) over the last two years are: 
416092 Mapleton Hill Boulder Boulder with percentage increase 18.87%
416085 East Aurora Boulder Boulder with percentage increase 18.69%
416088 University Hill Boulder Boulder with percentage increase 17.86%
416076 East Chautauqua Boulder Boulder with percentage increase 16.75%
416099 Newlands Boulder Boulder with percentage increase 13.45%


I would suggest she invest in the fastest growing neighborhood in the past 24 months, Mapleton Hill which had an ROI in the past 2 years of 19%. However, it does depend on her budget.


In [35]:
# 5. How much more money could this investor make if she were willing to invest in other markets in Colorado? 

# Running out of time.
# With more time, I would research time-series analysis in the statistics package
# Copying the code from #2 
months_lookback = 24
increases = {}
for k, v in colorado_residence_data.items():
    increases[k] = (v[-1]-v[-months_lookback])/v[-months_lookback]
    
# top five increases
find_increases = list(increases.values())
find_increases.sort(reverse=True)
max_increases = find_increases[0:5]

max_increase_regions = []
print('The top 5 neighborhoods in Colorado that have the largest increases (%) over the last two years are: ')
for i in max_increases:
    n = list(increases.values()).index(i)
    pct = '{0:.4g}'.format(i*100)
    print(f'{colorado_residence_iloc[n]} with percentage increase {pct}%')

print("\n")
print("This shows that in the past 24 months, the neighborhoods with the top 5 ROI in Colorado were not in Boulder, and she could make up to 14% more (34%-19%) by diversifying out of Boulder.")
 

The top 5 neighborhoods in Colorado that have the largest increases (%) over the last two years are: 
268633 Chaffee Park Denver Denver with percentage increase 34.31%
268659 Elyria Swansea Denver Denver with percentage increase 34.2%
268768 Villa Park Denver Denver with percentage increase 34.14%
268614 Barnum Denver Denver with percentage increase 32.47%
403509 College View - South Platte Denver Denver with percentage increase 32.39%


This shows that in the past 24 months, the neighborhoods with the top 5 ROI in Colorado were not in Boulder, and she could make up to 14% more (34%-19%) by diversifying out of Boulder.
